In [2]:
import pandas as pd
import numpy as np

This Jupyter notebook will be the Machine learning part of the project. We will import the clean data and inspect it again to check which columns will help us build our machine learning model. This will include preprocessing the data, encoding categorical values, scaling numerical ones and using them for clustering using k-means.

In [ ]:
# Load the dataset
df = pd.read_csv('../data/cleaned_data/student_social_media_addiction_cleaned.csv')

In [ ]:
# Inspect the data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 702 entries, 0 to 701
Data columns (total 16 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   new_student_id                702 non-null    object 
 1   age                           702 non-null    int64  
 2   gender                        702 non-null    object 
 3   relationship_status           702 non-null    object 
 4   academic_level                702 non-null    object 
 5   country                       702 non-null    object 
 6   continents                    702 non-null    object 
 7   most_used_platform            702 non-null    object 
 8   average_daily_usage_minutes   702 non-null    float64
 9   average_daily_use_by_hour     702 non-null    int64  
 10  sleep_per_night_minutes       702 non-null    float64
 11  sleep_hours_per_night         702 non-null    float64
 12  conflicts_over_social_media   702 non-null    int64  
 13  affec

In [ ]:
# Check the columns and pick out which ones we will use going forward
df.columns

Index(['new_student_id', 'age', 'gender', 'relationship_status',
       'academic_level', 'country', 'continents', 'most_used_platform',
       'average_daily_usage_minutes', 'average_daily_use_by_hour',
       'sleep_per_night_minutes', 'sleep_hours_per_night',
       'conflicts_over_social_media', 'affects_academic_performance',
       'addicted_score', 'mental_health_score'],
      dtype='object')

We will using the following columns:
- gender  
- relationship_status  
- academic_level  
- continents  
- average_daily_usage_minutes  
- sleep_per_night_minutes  
- conflicts_over_social_media  
- affects_academic_performance  
- addicted_score  
- mental_health_score
